# EDA - Bases de données patient, consommant et hospitalisation

### Installation libraire requise

In [ ]:
!pip install pandas_profiling

### Import librairies

In [ ]:
import numpy as np
import os
import pandas as pd
from pandas_profiling import ProfileReport

### Import des bases de données

In [ ]:
hospit=pd.read_csv("base_hospit.csv", sep=';')
patient=pd.read_csv("base_patient.csv", sep=';')
consommant=pd.read_csv("consommant.csv", sep=';')

### Conversion des dates en datetime

In [ ]:
hospit['EXE_SOI_DTD']=pd.to_datetime(hospit['EXE_SOI_DTD'], dayfirst=True)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].fillna(np.nan).astype(float)
hospit['SEJ_NBJ']=hospit['SEJ_NBJ'].astype('Int64')

In [ ]:
consommant['datemax']=pd.to_datetime(consommant['datemax'], dayfirst=True)

In [ ]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)
patient['dte_deces']=pd.to_datetime(patient['dte_deces'], dayfirst=True)

## Suppression des patients dont la première hospitalisation est postérieure à 2016 ou antérieure à 2010

In [ ]:
patient=patient[(patient['date_h0']<'01-01-2016')&(patient['date_h0']>='01-01-2010')]

In [ ]:
patient['date_h0']=pd.to_datetime(patient['date_h0'], dayfirst=True)

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
consommant.head()

In [ ]:
hospit.head()

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit.info()

## On conserve uniquement les mois de décès

In [ ]:
patient['dte_deces']=patient['dte_deces'].dt.to_period('M')

In [ ]:
patient.head()

## On conserve uniquement les patients majeurs au moment de leur première hospitalisation

#### Nombre de patients mineurs dans les données

In [ ]:
np.unique((patient['date_h0'].dt.to_period('Y')-patient['y_nais']).astype("str")>str(18), return_counts=True)

In [ ]:
patient = patient[(patient['date_h0'].dt.to_period('Y')-patient['y_nais']).astype("str")>str(18)]

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

## On retire les patients ayant déjà connu une hospitalisation avant l'étude (pop=0)

In [ ]:
patient=patient[patient["pop"]==1].reset_index(drop=True)

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

## On retire les patients sans consommation depuis plus de 5 ans, non déclarés décédés

In [ ]:
patient_non_décédés=patient[patient['dte_deces'].fillna(0)==0]["CODE_PATIENT"]

In [ ]:
patient_non_décédés

In [ ]:
patients_sans_conso=consommant[(consommant["CODE_PATIENT"].isin(patient_non_décédés))&(consommant['datemax']<'01-01-2014')]['CODE_PATIENT']

In [ ]:
patient = patient[~patient["CODE_PATIENT"].isin(patients_sans_conso)].reset_index(drop=True)

In [ ]:
consommant = consommant[consommant['CODE_PATIENT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

In [ ]:
hospit = hospit[hospit['BEN_NIR_IDT'].isin(patient['CODE_PATIENT'])].reset_index(drop=True)

## Ajout des insuffisances cardiaques avec état de choc cardiogénique

In [ ]:
#Insuffisances cardiaques et états de choc circulatoire
#hospit[hospit['GRG_GHM'].str.startswith('05M09')]

#Insuffisances cardiaques et états de choc cardiogénique (code R57)
#hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))]

#Ajout d'une colonne CHOC
hospit['CHOC']=hospit.index.isin(hospit[(hospit['GRG_GHM'].str.startswith('05M09'))&(hospit['DGN_PAL'].str.startswith('R57'))|(hospit['DGN_REL'].str.startswith('R57'))].index)
hospit['CHOC']=hospit['CHOC'].astype('int')

In [ ]:
hospit.CHOC.sum()

In [ ]:
hospit.info()

## Ajout des hémodialyses

In [ ]:
hospit['HEMOD']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z49'))].index)
hospit['HEMOD']=hospit['HEMOD'].astype('int')

In [ ]:
hospit['DGN_PAL'].str.startswith('Z49').sum()

## Ajout des chimiothérapies

In [ ]:
hospit['CHIMIO']=hospit.index.isin(hospit[(hospit['DGN_PAL'].str.startswith('Z511'))|(hospit['DGN_PAL'].str.startswith('Z512'))].index)
hospit['CHIMIO']=hospit['CHIMIO'].astype('int')

In [ ]:
hospit['DGN_PAL'].str.startswith('Z511').sum(), hospit['DGN_PAL'].str.startswith('Z512').sum()

In [ ]:
hospit

### Suppression de la colonne département de résidence des patients

In [ ]:
patient=patient.drop("BEN_RES_DPT", axis=1)

## Statistiques descriptives sur les datasets

### Hospitalisation

In [ ]:
hospit.info()

In [ ]:
profile_hospit = ProfileReport(hospit, title="Hospit Profile Report")

In [ ]:
profile_hospit.to_notebook_iframe()

In [ ]:
profile_hospit.to_file("profile_hospit.html")

### Consommant

In [ ]:
consommant.head()

In [ ]:
consommant.info()

In [ ]:
profile_conso = ProfileReport(consommant, title="Consommant Profile Report")

In [ ]:
profile_conso.to_notebook_iframe()

In [ ]:
profile_conso.to_file("profile_conso.html")

### Patient

In [ ]:
patient.head()

In [ ]:
patient.info()

In [ ]:
profile_patient = ProfileReport(patient, title="Patient Profile Report")

In [ ]:
profile_patient.to_notebook_iframe()

In [ ]:
profile_patient.to_file("profile_patient.html")

## Export des bases de données nettoyées

In [ ]:
hospit.to_csv("base_hospit_cleaned.csv", index=False)

In [ ]:
patient.to_csv("patient_cleaned.csv", index=False)

In [ ]:
consommant.to_csv("consommant_cleaned.csv", index=False)